In [2]:
# from kubernetes_asyncio import client, config
import kubernetes_asyncio as kube
import asyncio

In [3]:
async def list_pods_once():
	await kube.config.load_kube_config()
	api = kube.client.CoreV1Api()
	
	pod_list_response = await api.list_namespaced_pod('cvlab')
	
	print(' '.join(f'{pod.metadata.name}' for pod in pod_list_response.items))
	
	return pod_list_response
	
# asyncio.run(main())
pod_list = await list_pods_once()

# Configs can be set in Configuration class directly or using helper
# utility. If no argument provided, the config will be loaded from
# default location.


bedn bedn2 bedn3 benlalahpod.com guillard-sinkhorn-gridnet katircio-imagenet katircio-imagenet2 kicirogl-airsim-1 kicirogl-airsim-2 mv-cvpr3 nakka-advseg11 nakka-advseg14 nakka-advseg20 nsd oner-rt-fulldata-40 oner-rt-fulldata-40-2 rbermude-multiflow remelli-mva remelli-mvp sguo-pytorch-imagenet-iclr vidit-maskrcnn-seg-res50-2 wickrama-experiments-1 wickrama-experiments-2 wickrama-experiments-3 wwang-imagenet0 wwang-imagenet1 wwang-imagenet2


In [4]:
pod_to_test = pod_list.items[0]
print(f'Testing on pod [{pod_to_test.metadata.name}]')

Testing on pod [bedn]


### GPU status with nvidia-smi

Example queries: <https://nvidia.custhelp.com/app/answers/detail/a_id/3751/~/useful-nvidia-smi-queries>  
List of fields: <https://briot-jerome.developpez.com/fichiers/blog/nvidia-smi/list.txt>

`nvidia-smi --format=csv --loop=2 --query-gpu=index,utilization.gpu,utilization.memory,memory.used,memory.total`

Injections:  
`kubectl exec -it container -- /usr/bin/nvidia-smi --format=csv --loop=2 --query-gpu=index,utilization.gpu,utilization.memory,memory.used,memory.total`

Run command for 10s:  
`timeout 10 something`

In [5]:
api = kube.client.CoreV1Api()
api.connect_get_namespaced_pod_exec?

Signature: api.connect_get_namespaced_pod_exec(name, namespace, **kwargs)
Docstring:
connect_get_namespaced_pod_exec  # noqa: E501

connect GET requests to exec of Pod  # noqa: E501
This method makes a synchronous HTTP request by default. To make an
asynchronous HTTP request, please pass async_req=True
>>> thread = api.connect_get_namespaced_pod_exec(name, namespace, async_req=True)
>>> result = thread.get()

:param async_req bool
:param str name: name of the PodExecOptions (required)
:param str namespace: object name and auth scope, such as for teams and projects (required)
:param str command: Command is the remote command to execute. argv array. Not executed within a shell.
:param str container: Container in which to execute the command. Defaults to only container if there is only one container in the pod.
:param bool stderr: Redirect the standard error stream of the pod for this call. Defaults to true.
:param bool stdin: Redirect the standard input stream of the pod for this call. D

### Executing inside a container

Example: <https://github.com/tomplus/kubernetes_asyncio/blob/master/examples/example3.py>

In [12]:
GPU_QUERY_FIELDS = [
	'index',
	'utilization.gpu',
	'utilization.memory',
	'memory.used',
	'memory.total',
]
GPU_QUERY_CMD = [
	'/usr/bin/timeout', '5',
	'/usr/bin/nvidia-smi',
	'--format=csv',
  	'--loop=1',
	f'--query-gpu={",".join(GPU_QUERY_FIELDS)}',
]
print(GPU_QUERY_CMD)

async def query_gpu_status():
	await kube.config.load_kube_config()
	api_ws = kube.client.CoreV1Api(api_client=kube.stream.WsApiClient())
# 	api_ws = kube.client.CoreV1Api()

	cmd = GPU_QUERY_CMD
	name = 'sguo-pytorch-imagenet-iclr'
# 	name = 'wwang-imagenet2'
	namespace = 'cvlab'
	
# 	cmd = ['ls']
	
	req = api_ws.connect_get_namespaced_pod_exec(
		name = name, 
		namespace = namespace,
		command = cmd,
		stderr=True,
		stdin=False,
		stdout=True,
		tty=False,

# 		async_req=True,
# 		_preload_content=False,
	)
	
	print('Req', req)
	response = await req
	print("Query response: ", response)
	return response
	
# 	w = kube.watch.Watch()
	
# 	async for event in w.stream(api_ws.connect_get_namespaced_pod_exec,
# 		name = name, 
# 		namespace = namespace,
# 		command = cmd,
# 		stderr=True,
# 		stdin=False,
# 		stdout=True,
# 		tty=False,
								
# 		async_req=True,
# 		_preload_content=False,
# 	):
# 		print(event)
	

# 	return response

r = await query_gpu_status()

['/usr/bin/timeout', '5', '/usr/bin/nvidia-smi', '--format=csv', '--loop=1', '--query-gpu=index,utilization.gpu,utilization.memory,memory.used,memory.total']
Req <coroutine object ApiClient.__call_api at 0x7f7c278533b0>
Query response:  index, utilization.gpu [%], utilization.memory [%], memory.used [MiB], memory.total [MiB]
0, 99 %, 60 %, 17613 MiB, 32510 MiB
1, 100 %, 60 %, 17547 MiB, 32510 MiB
0, 99 %, 58 %, 17613 MiB, 32510 MiB
1, 100 %, 56 %, 17547 MiB, 32510 MiB
0, 99 %, 66 %, 17613 MiB, 32510 MiB
1, 100 %, 65 %, 17547 MiB, 32510 MiB
0, 99 %, 60 %, 17613 MiB, 32510 MiB
1, 100 %, 59 %, 17547 MiB, 32510 MiB
0, 100 %, 65 %, 17613 MiB, 32510 MiB
1, 100 %, 67 %, 17547 MiB, 32510 MiB



In [13]:
import csv

In [22]:
list(csv.reader(r.split('\n')))

[['index',
  ' utilization.gpu [%]',
  ' utilization.memory [%]',
  ' memory.used [MiB]',
  ' memory.total [MiB]'],
 ['0', ' 99 %', ' 60 %', ' 17613 MiB', ' 32510 MiB'],
 ['1', ' 100 %', ' 60 %', ' 17547 MiB', ' 32510 MiB'],
 ['0', ' 99 %', ' 58 %', ' 17613 MiB', ' 32510 MiB'],
 ['1', ' 100 %', ' 56 %', ' 17547 MiB', ' 32510 MiB'],
 ['0', ' 99 %', ' 66 %', ' 17613 MiB', ' 32510 MiB'],
 ['1', ' 100 %', ' 65 %', ' 17547 MiB', ' 32510 MiB'],
 ['0', ' 99 %', ' 60 %', ' 17613 MiB', ' 32510 MiB'],
 ['1', ' 100 %', ' 59 %', ' 17547 MiB', ' 32510 MiB'],
 ['0', ' 100 %', ' 65 %', ' 17613 MiB', ' 32510 MiB'],
 ['1', ' 100 %', ' 67 %', ' 17547 MiB', ' 32510 MiB'],
 []]